In [ ]:
!nvidia-smi

In [1]:
%matplotlib inline

# uso y pruebas

In [2]:
!python classifier.py --help
# !python classifier_bert.py --help
# !python classifier.py --help

usage: classifier.py [-h] [-g GPU] [-f MODEL_FILE] [-t TRAIN_CFG]
                     [-a MASK_CFG] [-l LABEL_INDEX] [-s SAVE_MODEL]
                     model_version {hhar,motion,uci,shoaib} {10_100,20_120}

PyTorch LIMU-BERT Model

positional arguments:
  model_version         Model config
  {hhar,motion,uci,shoaib}
                        Dataset name
  {10_100,20_120}       Dataset version

optional arguments:
  -h, --help            show this help message and exit
  -g GPU, --gpu GPU     Set specific GPU
  -f MODEL_FILE, --model_file MODEL_FILE
                        Pretrain model file
  -t TRAIN_CFG, --train_cfg TRAIN_CFG
                        Training config json file path
  -a MASK_CFG, --mask_cfg MASK_CFG
                        Mask strategy json file path
  -l LABEL_INDEX, --label_index LABEL_INDEX
                        Label Index
  -s SAVE_MODEL, --save_model SAVE_MODEL
                        The saved model name
usage: classifier.py [-h] [-g GPU] [-f MODEL_FILE] 

In [3]:
!python embedding.py v1 uci 20_120 -f uci

cuda:0 (1 GPUs)
Loading the model from saved/pretrain_base_uci_20_120/uci
Traceback (most recent call last):
  File "embedding.py", line 64, in <module>
    data, output, labels = generate_embedding_or_output(args=args, output_embed=True, save=save)
  File "embedding.py", line 48, in generate_embedding_or_output
    np.save(os.path.join('embed', save_name + '.npy'), output)
  File "<__array_function__ internals>", line 5, in save
  File "/home/baruj/miniconda3/envs/limu-py38/lib/python3.8/site-packages/numpy/lib/npyio.py", line 541, in save
    fid = open(file, "wb")
FileNotFoundError: [Errno 2] No such file or directory: 'embed/embed_uci_uci_20_120.npy'


In [5]:
!pip install -r requirements.txt

  Using cached numpy-1.18.5-cp38-cp38-manylinux1_x86_64.whl.metadata (2.1 kB)
Using cached numpy-1.18.5-cp38-cp38-manylinux1_x86_64.whl (20.6 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.3
    Uninstalling numpy-1.24.3:
      Successfully uninstalled numpy-1.24.3


In [9]:

!pip install tensorboard

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 7.3 MB/s eta 0:00:0000:0100:01


In [10]:
!python pretrain.py v1 uci 20_120 -s limu_v1 

Train Size: 1670, Vali Size: 208, Test Size: 210
cuda:0 (1 GPUs)
Epoch 1/3200 : Average Loss 0.1521. Test Loss 0.0893
Epoch 2/3200 : Average Loss 0.0747. Test Loss 0.0788
Epoch 3/3200 : Average Loss 0.0757. Test Loss 0.0695
Epoch 4/3200 : Average Loss 0.0638. Test Loss 0.0699
Epoch 5/3200 : Average Loss 0.0628. Test Loss 0.0704
Epoch 6/3200 : Average Loss 0.0636. Test Loss 0.0672
Epoch 7/3200 : Average Loss 0.0647. Test Loss 0.0649
Epoch 8/3200 : Average Loss 0.0634. Test Loss 0.0647
Epoch 9/3200 : Average Loss 0.0617. Test Loss 0.0676
Epoch 10/3200 : Average Loss 0.0612. Test Loss 0.0660
Epoch 11/3200 : Average Loss 0.0622. Test Loss 0.0675
Epoch 12/3200 : Average Loss 0.0593. Test Loss 0.0677
Epoch 13/3200 : Average Loss 0.0617. Test Loss 0.0669
Epoch 14/3200 : Average Loss 0.0660. Test Loss 0.0668
Epoch 15/3200 : Average Loss 0.0647. Test Loss 0.0672
Epoch 16/3200 : Average Loss 0.0591. Test Loss 0.0686
Epoch 17/3200 : Average Loss 0.0611. Test Loss 0.0690
Epoch 18/3200 : Average Lo

In [ ]:
!python embedding.py v1 uci 20_120 -f limu_v1


In [ ]:
!python embedding.py "BERT" "hhar" "20_120" -g 0

# utils.py linea 411 msj de erro

# carga desede codigo gpt

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd
import os
import json
import plot 
import utils 
import torch
import torch.nn as nn

In [ ]:
class LIMUBERTModel(nn.Module):
    def __init__(self):
        super(LIMUBERTModel, self).__init__()
        self.embed = nn.Linear(6, 72)  # Adjusted to match the saved model
        self.transformer = nn.Transformer(d_model=72, nhead=8, num_encoder_layers=6)  # Adjusted to match the saved model
        self.fc = nn.Linear(72, 72)  # Adjusted to match the saved model

    def forward(self, x):
        x = self.embed(x)
        x = self.transformer(x)
        x = self.fc(x)
        return x

    def get_embeddings(self, x):
        x = self.embed(x)
        return x

# Instantiate the model
model = LIMUBERTModel()

# Path to your model file
model_path = 'saved/pretrain_base_hhar_20_120/hhar.pt'

# Load the state dictionary
state_dict = torch.load(model_path)

# Load the state dict into the model
model.load_state_dict(state_dict, strict=False)

# Set model to evaluation mode
model.eval()


In [ ]:
# Load dataset
data_path = "dataset/hhar/data_20_120.npy"
labels_path = 'dataset/hhar/label_20_120.npy'

data = np.load(data_path)
labels = np.load(labels_path)

# Flatten the data
data_flat = data.reshape(data.shape[0], -1)



# Extract embeddings from the flattened data
embeddings = get_embeddings(model, data_flat)

# Ensure labels are correctly shaped
if len(labels.shape) > 1 and labels.shape[1] > 1:
    labels = labels[:, 0]

# Function to visualize embeddings
def visualize_embeddings(embeddings, labels):
    tsne = TSNE(n_components=2, random_state=42)
    reduced_embeddings = tsne.fit_transform(embeddings)
    
    plt.figure(figsize=(10, 10))
    scatter = plt.scatter(reduced_embeddings[:, 0], reduced_embeddings[:, 1], c=labels, cmap='viridis', alpha=0.5)
    plt.colorbar(scatter)
    plt.title("t-SNE visualization of embeddings")
    plt.xlabel("t-SNE component 1")
    plt.ylabel("t-SNE component 2")
    plt.show()

# Inline plotting for Jupyter Notebook
%matplotlib inline

# Visualize the embeddings
visualize_embeddings(state_dict, labels)
